In [46]:
import pandas as pd
import numpy as np
import warnings
import sys
from lgbm_process import lgbm_process
sys.path.append("C:/Users/User/Desktop/LG Aimers")  # LG_Aimers_6th의 상위 폴더

from LG_Aimers_6th.cal_auc import calculate_auc


warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

## Random Forest

In [47]:
data_seed = 9
folds = 5 ####

In [48]:
train_path = f'../data/custom_train_{data_seed}.csv'
test_path = f'../data/custom_test_{data_seed}.csv'

train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])
train, test = lgbm_process(train, test)
print(train.shape, test.shape)

(205080, 67) (51271, 66)


In [49]:
train.head()

,시술 시기 코드,시술 당시 나이,임신 시도 또는 마지막 임신 경과 연수,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 검사 사용 여부,착상 전 유전 진단 사용 여부,남성 주 불임 원인,남성 부 불임 원인,...,난자 혼합 경과일,배아 이식 경과일,배아 해동 경과일,임신 성공 여부,시술유형_통합,시술_임신,배아생성이유_기증용,배아생성이유_난자 저장용,배아생성이유_배아 저장용,배아생성이유_현재 시술용
0,5.0,0.0,0.0,1,0.0,1.0,0.0,0.0,0,0,...,0.0,5.0,0.0,0,5.0,-1,0.0,0.0,0.0,1.0
1,3.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0,0,...,0.0,6.0,0.0,0,6.0,-2,0.0,0.0,0.0,1.0
2,5.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0,0,...,0.0,3.0,0.0,1,5.0,0,0.0,0.0,0.0,1.0
3,3.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0,0,...,0.0,3.0,0.0,1,5.0,-1,0.0,0.0,0.0,1.0
4,1.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0,6.0,-3,0.0,0.0,0.0,1.0


In [50]:
train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

total_auc, total_acc, total_f1 = [], [], []
test_preds_rf = []

seed = 333
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
auc_scores, acc_scores,  f1_scores = [], [], []

for fold, (train_idx, valid_idx) in enumerate(skf.split(train.drop(columns=['임신 성공 여부']), train["임신 성공 여부"])):
    fold_train = train.iloc[train_idx].copy().reset_index(drop=True)
    fold_train2 = train.iloc[train_idx].copy().reset_index(drop=True)
    fold_valid = train.iloc[valid_idx].copy().reset_index(drop=True)
    fold_test = test.copy()

    fold_train, fold_valid = lgbm_process(fold_train, fold_valid, seed=seed)
    fold_train2, fold_test = lgbm_process(fold_train2, fold_test, seed=seed)

    X_train = fold_train.drop(columns=['임신 성공 여부'])
    X_valid = fold_valid.drop(columns=['임신 성공 여부'])
    y_train = fold_train['임신 성공 여부']
    y_valid = fold_valid['임신 성공 여부']


    
    # GPT 추천 파라미터
    rf_params = {
        'n_estimators': 300,         # ⭐ [추천] 기본보다 늘려서 안정적인 앙상블 효과 확보.
                                    # 너무 많으면 시간 증가, 300~500 선에서 안정적 성능 기대.

        'criterion': 'gini',         # ⭐ [추천] 대부분의 경우 'gini'가 속도 빠르고 성능 차이 미미.
                                    # 불균형 데이터면 'log_loss' 시도해볼 가치 있음.

        'max_depth': 20,             # ⭐ [추천] 트리 깊이를 제한하여 과적합 방지.
                                    # 데이터가 많으므로 지나치게 깊게 갈 필요 없음.

        'min_samples_split': 10,     # ⭐ [추천] 노드 분할을 더 보수적으로.
                                    # 샘플 수가 많을수록 기본값보다 높게 설정하는 게 안정적.

        'min_samples_leaf': 4,       # ⭐ [추천] 리프 노드 최소 샘플 수 증가로 과적합 방지.
                                    # leaf 수 증가 = 더 일반화된 분기.

        'max_features': 'sqrt',      # [유지] 분산 낮추고 속도 개선 → 기본값 유지 추천.

        'max_leaf_nodes': None,      # [옵션] 성능이 잘 나오면 무제한으로 두고, 과적합 의심 시 100~500 제한.

        'min_impurity_decrease': 0.0,# [유지] 기본으로 충분. 필요 시 미세 튜닝 가능.

        'bootstrap': True,           # [유지] 부트스트랩 샘플링 사용 → Random Forest 본연의 방식.

        'random_state': seed,        # [유지] 결과 재현 가능하도록.

        'verbose': 0,                # [유지] 학습 로그 불필요 시 유지.

        'warm_start': False,         # [유지] 반복 학습 필요 없으면 False 유지.

        'class_weight': 'balanced',  # ⭐ [추천] 클래스 불균형이 있다면 자동으로 보정.
                                    # 특히 '임신 성공 여부'처럼 비율 차이 큰 경우 성능 향상 가능.

        'ccp_alpha': 0.0,            # [유지] 기본적으로는 0.0 사용, 과적합 감지 시 0.0001 정도 시도.

        'max_samples': 0.7           # ⭐ [추천] 학습 샘플 일부만 사용하는 방식 (부트스트랩)
                                    # 속도 단축 및 다소의 regularization 효과 있음.
    }

    model_rf = RandomForestClassifier(**rf_params)

    model_rf.fit(X_train, y_train) # 파라미터 확인 eval set이 있을 수도도

    valid_preds_proba  = model_rf.predict_proba(X_valid)[:, 1]
    valid_preds_class = model_rf.predict(X_valid)  # 클래스 예측값 (0 또는 1)

    # AUC, Accuracy, F1-score 계산
    auc_ = roc_auc_score(y_valid, valid_preds_proba)
    acc_ = accuracy_score(y_valid, valid_preds_class)
    f1_ = f1_score(y_valid, valid_preds_class)

    print(f"Seed[{seed:<3}] Fold {fold + 1} | AUC: {auc_:.7f} | Acc: {acc_:.7f} | F1: {f1_:.7f}")

    auc_scores.append(auc_)
    acc_scores.append(acc_)
    f1_scores.append(f1_)

    total_auc.append(auc_)
    total_acc.append(acc_)
    total_f1.append(f1_)

    test_pred = model_rf.predict_proba(fold_test)[:, 1]
    test_preds_rf.append(test_pred)

# fold 별 평균 성능 계산
avg_auc = np.mean(auc_scores)
avg_acc = np.mean(acc_scores)
avg_f1 = np.mean(f1_scores)

print("-" * 80)
print(f"Seed[{seed:<3}] Average Metrics | AUC: {avg_auc:.7f} | Acc: {avg_acc:.7f} | F1: {avg_f1:.7f}")

Seed[333] Fold 1 | AUC: 0.7364401 | Acc: 0.6521845 | F1: 0.5096583
Seed[333] Fold 2 | AUC: 0.7283998 | Acc: 0.6465769 | F1: 0.5046135
Seed[333] Fold 3 | AUC: 0.7351564 | Acc: 0.6539155 | F1: 0.5101625


c:\Users\User\anaconda3\envs\lg2025\lib\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['연구용'] will be ignored
  warnings.warn(
c:\Users\User\anaconda3\envs\lg2025\lib\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['연구용'] will be ignored
  warnings.warn(


Seed[333] Fold 4 | AUC: 0.7325390 | Acc: 0.6509411 | F1: 0.5061230
Seed[333] Fold 5 | AUC: 0.7340557 | Acc: 0.6522333 | F1: 0.5084092
--------------------------------------------------------------------------------
Seed[333] Average Metrics | AUC: 0.7333182 | Acc: 0.6511703 | F1: 0.5077933


In [51]:
tmp_submission = pd.DataFrame({f'RandomForest_{data_seed}': np.mean(test_preds_rf, axis=0)})
tmp_submission

,RandomForest_9
0,0.493770
1,0.526425
2,0.000165
3,0.154574
4,0.588265
...,...
51266,0.648031
51267,0.446124
51268,0.339861
51269,0.512979


In [52]:
# Save the DataFrame to a CSV file
file_name = f"Random Forest_{data_seed}_result.csv"
tmp_submission.to_csv(file_name, index=False)

In [53]:
score = calculate_auc(tmp_submission, seed=data_seed)
score

0.7379787082709299